# <div align="center"><b> Análisis de resultados - EuroSAT </b></div>

<div align="right">

<!-- [![Binder](http://mybinder.org/badge.svg)](https://mybinder.org/) -->
[![nbviewer](https://img.shields.io/badge/render-nbviewer-orange?logo=Jupyter)](https://nbviewer.org/)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

</div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻


Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:

```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:

```powershell
poetry install
Invoke-Expression (poetry env activate)
```

> 📝 <em><font color='Gray'>Nota:</font></em> Para agregar `pytorch` utilizando Poetry, se utiliza el siguiente comando:
> ```bash
> # Más info: https://github.com/python-poetry/poetry/issues/6409
> potery source add --priority explicit pytorch_gpu https://download.pytorch.org/whl/cu128 # Seleccionar la wheel adecuada para tu GPU
> poetry add --source pytorch_gpu torch torchvision 
> ```

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [ ]:
# Recarga automática de módulos en Jupyter Notebook
%reload_ext autoreload
%autoreload 2

import pandas as pd
import plotly.graph_objects as go

# Modulos propios
from vision_transformer.config import MODELS_DIR, MODELS_DIR_SIWNV2_TINY, MODELS_DIR_SIWNV2_BASE, MODELS_DIR_SIWNV2_LARGE, METRICS_FILENAME, REPORTS_DIR

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Análisis de resultados de entrenamiento de los modelos sobre EuroSAT                                                                       |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | <small>Análisis de resultados sobre el EuroSAT<br/>- *Tarea:* `Clasificación`<br/>- *Modelos*: `swinV2-tiny`, `swinV2-base`, `swinV2-large` <br/> - *Dataset*: `EuroSAT` </small>|
<!-- | **Autor** | <small>[Nombre] ([correo]) </small>                                                                                                 | -->

</div>

## Tabla de contenidos
1. [Carga de datos](#carga-de-datos)

In [7]:
swinv2_tiny_metrics = pd.read_csv(MODELS_DIR_SIWNV2_TINY / METRICS_FILENAME)
swinv2_base_metrics = pd.read_csv(MODELS_DIR_SIWNV2_BASE / METRICS_FILENAME)
swinv2_large_metrics = pd.read_csv(MODELS_DIR_SIWNV2_LARGE / METRICS_FILENAME)

## 2. [Análisis de resultados](#análisis-de-resultados)

### 2.1. [SwinV2](#swinv2)

In [ ]:
# Add a 'Model' column to each dataframe
swinv2_tiny_metrics['Model'] = 'SwinV2-Tiny'
swinv2_base_metrics['Model'] = 'SwinV2-Base'
swinv2_large_metrics['Model'] = 'SwinV2-Large'

# Concatenate the dataframes
all_metrics = pd.concat([swinv2_tiny_metrics, swinv2_base_metrics, swinv2_large_metrics], ignore_index=True)

# Define the metrics to be plotted (excluding 'epoch' as it's a fixed value)
metrics = ['eval_loss', 'eval_accuracy', 'eval_f1', 'eval_precision', 'eval_recall',
           'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second']

# Create the radar chart
fig = go.Figure()

for model in all_metrics['Model'].unique():
    model_data = all_metrics[all_metrics['Model'] == model].iloc[0] # Get the single row of metrics for the model
    values = [model_data[metric] for metric in metrics]
    # In order to close the radar chart, the first value must be repeated at the end
    values = values + [values[0]]
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=metrics + [metrics[0]], # Repeat the first metric to close the shape
        fill='toself',
        name=model
    ))

fig.update_layout(
    polar=dict(
        radialaxis_visible=True,
        radialaxis=dict(
            gridcolor="rgba(0, 0, 0, 0.2)",
            linecolor="rgba(0, 0, 0, 0.2)",
            tickfont_color="rgba(0, 0, 0, 0.8)",
        ),
        angularaxis=dict(
            linewidth=1,
            linecolor='gray',
            gridcolor="rgba(0, 0, 0, 0.2)",
            tickfont_color="rgba(0, 0, 0, 0.8)",
            ticklen=10,
            tickfont_size=10,
        )
    ),
    title="Comparación de Métricas de Modelos SwinV2",
    title_x=0.5, # Center the title
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()
fig.write_json(REPORTS_DIR / 'swinv2_radar_chart.json')

In [ ]:
# https://plotly.com/python/radar-chart/


df = pd.DataFrame(dict(
    r=[1, 5, 2, 2, 3],
    theta=['processing cost','mechanical properties','chemical stability',
           'thermal stability', 'device integration']))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.show()